In [1]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
from shapely.geometry import Point, Polygon, LineString
import matplotlib as mpl
import pickle

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
build_road=gpd.read_file('15.도로명주소_건물.geojson')
code_excel = pd.read_excel('28.코드정의서.xlsx') # 28코드정의서 엑셀 파일

In [ ]:
build_road

,BDTYP_CD,BULD_NM,BULD_NM_DC,EMD_CD,GRO_FLO_CO,SIG_CD,UND_FLO_CO,geometry
0,01003,None,None,105,4,41281,0,"MULTIPOLYGON (((126.87256 37.62531, 126.87259 ..."
1,02003,헤리티지,3단지,113,4,41285,1,"MULTIPOLYGON (((126.77481 37.70779, 126.77485 ..."
2,02003,헤리티지,2단지,113,4,41285,1,"MULTIPOLYGON (((126.77510 37.70767, 126.77515 ..."
3,03999,None,None,111,3,41281,0,"MULTIPOLYGON (((126.89398 37.65698, 126.89411 ..."
4,03005,None,None,103,5,41281,0,"MULTIPOLYGON (((126.89124 37.66385, 126.89140 ..."
...,...,...,...,...,...,...,...,...
79997,03012,None,None,132,1,41281,0,"MULTIPOLYGON (((126.88868 37.59669, 126.88878 ..."
79998,21999,None,None,132,1,41281,0,"MULTIPOLYGON (((126.88879 37.59669, 126.88889 ..."
79999,03001,None,None,115,1,41281,0,"MULTIPOLYGON (((126.90831 37.69544, 126.90847 ..."
80000,01003,None,None,132,4,41281,0,"MULTIPOLYGON (((126.89266 37.60498, 126.89280 ..."


In [ ]:
#28.코드정의서에서 건축물용도코드(BDTYP_CD)만 가져오기
BDTYP_CD_meaning = code_excel[64:537]
BDTYP_CD_meaning.columns = ['BDTYP_CD','코드값 의미']
BDTYP_CD_meaning

,BDTYP_CD,코드값 의미
64,01000,단독주택
65,01001,단독주택
66,01002,다중주택
67,01003,다가구주택
68,01004,공관
...,...,...
532,28015,창고용천막
533,28016,관광문화행사용가설건축물
534,28999,기타가설건축물
535,29000,장례식장


In [ ]:
# 공간정보 데이터의 중심을 구하기
build_road['geometry_Point'] = build_road.geometry.centroid

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


In [ ]:
build_road

,BDTYP_CD,BULD_NM,BULD_NM_DC,EMD_CD,GRO_FLO_CO,SIG_CD,UND_FLO_CO,geometry,geometry_Point
0,01003,None,None,105,4,41281,0,"MULTIPOLYGON (((126.87256 37.62531, 126.87259 ...",POINT (126.87260 37.62525)
1,02003,헤리티지,3단지,113,4,41285,1,"MULTIPOLYGON (((126.77481 37.70779, 126.77485 ...",POINT (126.77496 37.70778)
2,02003,헤리티지,2단지,113,4,41285,1,"MULTIPOLYGON (((126.77510 37.70767, 126.77515 ...",POINT (126.77525 37.70766)
3,03999,None,None,111,3,41281,0,"MULTIPOLYGON (((126.89398 37.65698, 126.89411 ...",POINT (126.89405 37.65696)
4,03005,None,None,103,5,41281,0,"MULTIPOLYGON (((126.89124 37.66385, 126.89140 ...",POINT (126.89125 37.66371)
...,...,...,...,...,...,...,...,...,...
79997,03012,None,None,132,1,41281,0,"MULTIPOLYGON (((126.88868 37.59669, 126.88878 ...",POINT (126.88872 37.59662)
79998,21999,None,None,132,1,41281,0,"MULTIPOLYGON (((126.88879 37.59669, 126.88889 ...",POINT (126.88883 37.59665)
79999,03001,None,None,115,1,41281,0,"MULTIPOLYGON (((126.90831 37.69544, 126.90847 ...",POINT (126.90838 37.69538)
80000,01003,None,None,132,4,41281,0,"MULTIPOLYGON (((126.89266 37.60498, 126.89280 ...",POINT (126.89271 37.60491)


In [ ]:
build_road_plus = pd.merge(build_road,BDTYP_CD_meaning, on='BDTYP_CD', how='outer')
build_road_plus

,BDTYP_CD,BULD_NM,BULD_NM_DC,EMD_CD,GRO_FLO_CO,SIG_CD,UND_FLO_CO,geometry,geometry_Point,코드값 의미
0,01003,None,None,105,4.0,41281,0.0,"MULTIPOLYGON (((126.87256 37.62531, 126.87259 ...",POINT (126.87260 37.62525),다가구주택
1,01003,None,None,106,3.0,41285,0.0,"MULTIPOLYGON (((126.78974 37.64674, 126.78980 ...",POINT (126.78972 37.64668),다가구주택
2,01003,None,None,106,3.0,41281,0.0,"MULTIPOLYGON (((126.85353 37.64523, 126.85360 ...",POINT (126.85361 37.64514),다가구주택
3,01003,None,None,103,3.0,41285,0.0,"MULTIPOLYGON (((126.77427 37.67318, 126.77433 ...",POINT (126.77435 37.67319),다가구주택
4,01003,None,B동,107,4.0,41285,0.0,"MULTIPOLYGON (((126.79842 37.67501, 126.79854 ...",POINT (126.79848 37.67495),다가구주택
...,...,...,...,...,...,...,...,...,...,...
80331,28015,NaN,NaN,NaN,NaN,NaN,NaN,None,None,창고용천막
80332,28016,NaN,NaN,NaN,NaN,NaN,NaN,None,None,관광문화행사용가설건축물
80333,28999,NaN,NaN,NaN,NaN,NaN,NaN,None,None,기타가설건축물
80334,29000,NaN,NaN,NaN,NaN,NaN,NaN,None,None,장례식장


In [ ]:
build_road_plus = build_road_plus[:80002] # 컬럼 geometry에 NAN의 행은 제거해 준다.
build_road_plus

,BDTYP_CD,BULD_NM,BULD_NM_DC,EMD_CD,GRO_FLO_CO,SIG_CD,UND_FLO_CO,geometry,geometry_Point,코드값 의미
0,01003,None,None,105,4.0,41281,0.0,"MULTIPOLYGON (((126.87256 37.62531, 126.87259 ...",POINT (126.87260 37.62525),다가구주택
1,01003,None,None,106,3.0,41285,0.0,"MULTIPOLYGON (((126.78974 37.64674, 126.78980 ...",POINT (126.78972 37.64668),다가구주택
2,01003,None,None,106,3.0,41281,0.0,"MULTIPOLYGON (((126.85353 37.64523, 126.85360 ...",POINT (126.85361 37.64514),다가구주택
3,01003,None,None,103,3.0,41285,0.0,"MULTIPOLYGON (((126.77427 37.67318, 126.77433 ...",POINT (126.77435 37.67319),다가구주택
4,01003,None,B동,107,4.0,41285,0.0,"MULTIPOLYGON (((126.79842 37.67501, 126.79854 ...",POINT (126.79848 37.67495),다가구주택
...,...,...,...,...,...,...,...,...,...,...
79997,27999,None,None,104,5.0,41287,1.0,"MULTIPOLYGON (((126.73415 37.66356, 126.73435 ...",POINT (126.73438 37.66325),기타관광휴게시설
79998,90002,None,A30,123,0.0,41281,0.0,"MULTIPOLYGON (((126.83330 37.63768, 126.83327 ...",POINT (126.83328 37.63770),NaN
79999,90002,None,None,123,0.0,41281,0.0,"MULTIPOLYGON (((126.83208 37.63485, 126.83211 ...",POINT (126.83210 37.63484),NaN
80000,90002,None,C62,105,0.0,41287,0.0,"MULTIPOLYGON (((126.75660 37.69236, 126.75659 ...",POINT (126.75661 37.69237),NaN


In [ ]:
 build_road_plus['코드값 의미'].fillna('무',inplace=True) # 코득밧 의미 컬럼에 Nan값을 무로 바꿔준다. 'BDTYP_CD_코드값의미'를 생성하기위해

In [ ]:
build_road_plus['BDTYP_CD_코드값의미'] = build_road_plus[['BDTYP_CD','코드값 의미']].apply(lambda x:'_'.join(x),axis=1) # 새로운 컬럼 만들기

In [ ]:
build_road_plus

,BDTYP_CD,BULD_NM,BULD_NM_DC,EMD_CD,GRO_FLO_CO,SIG_CD,UND_FLO_CO,geometry,geometry_Point,코드값 의미,BDTYP_CD_코드값의미
0,01003,None,None,105,4.0,41281,0.0,"MULTIPOLYGON (((126.87256 37.62531, 126.87259 ...",POINT (126.87260 37.62525),다가구주택,01003_다가구주택
1,01003,None,None,106,3.0,41285,0.0,"MULTIPOLYGON (((126.78974 37.64674, 126.78980 ...",POINT (126.78972 37.64668),다가구주택,01003_다가구주택
2,01003,None,None,106,3.0,41281,0.0,"MULTIPOLYGON (((126.85353 37.64523, 126.85360 ...",POINT (126.85361 37.64514),다가구주택,01003_다가구주택
3,01003,None,None,103,3.0,41285,0.0,"MULTIPOLYGON (((126.77427 37.67318, 126.77433 ...",POINT (126.77435 37.67319),다가구주택,01003_다가구주택
4,01003,None,B동,107,4.0,41285,0.0,"MULTIPOLYGON (((126.79842 37.67501, 126.79854 ...",POINT (126.79848 37.67495),다가구주택,01003_다가구주택
...,...,...,...,...,...,...,...,...,...,...,...
79997,27999,None,None,104,5.0,41287,1.0,"MULTIPOLYGON (((126.73415 37.66356, 126.73435 ...",POINT (126.73438 37.66325),기타관광휴게시설,27999_기타관광휴게시설
79998,90002,None,A30,123,0.0,41281,0.0,"MULTIPOLYGON (((126.83330 37.63768, 126.83327 ...",POINT (126.83328 37.63770),무,90002_무
79999,90002,None,None,123,0.0,41281,0.0,"MULTIPOLYGON (((126.83208 37.63485, 126.83211 ...",POINT (126.83210 37.63484),무,90002_무
80000,90002,None,C62,105,0.0,41287,0.0,"MULTIPOLYGON (((126.75660 37.69236, 126.75659 ...",POINT (126.75661 37.69237),무,90002_무
